# Bokeh Tree draw example
*  Options:
  * user converting TTree to pandas + interactive drawing using pandas
  * using TTree draw using tree as input
  
* Demonstration of different widget types 
* TODO - show also:
  * layouts
  * for multigraph more suitable data to be used ...
  


In [ ]:
from RootInteractive.InteractiveDrawing.bokeh.bokehDraw import *
from bokeh.io import output_notebook, show
output_notebook()
p3 = figure(plot_width=700, plot_height=400, title="template")

# Load tree and define aliases (panda and bokeh does not accept arrays and . in variable names )

In [ ]:
ROOT.TFile.SetCacheFileDir("../data/")
tree=ROOT.AliTreePlayer.LoadTrees("cat performance.list","identFit","xxx",".*","","")
tree.SetAlias("norm","param.fElements[0]")
tree.SetAlias("slope","param.fElements[1]")
tree.SetAlias("isMax","name.String().Contains(\"PionMax\")==0");
tree.SetAlias("isTot","name.String().Contains(\"PionTot\")==0&&name.String().Contains(\"PionMaxTot\")==0");
tree.SetAlias("isMaxTot","name.String().Contains(\"PionMaxTot\")>0");
tree.SetAlias("P0","name.String().Contains(\"P0\")>0");
tree.SetAlias("P1","name.String().Contains(\"P1\")>0");
tree.SetAlias("PA","name.String().Contains(\"PA\")>0");
tree.SetAlias("MB","name.String().Contains(\"MB\")>0");

# Bokeh draw directly from the tree

In [ ]:
getTreeInfo(tree)

In [ ]:
tooltips=[('Is maximum','@isMax'),("Pad type","@typeF")]
tooltips[0][1]

In [ ]:
bokehDraw.verbosity=0
variables="pileUp:tailF:commonF:typeF:fraction:norm:slope:isMax:MB:P0:P1:PA"
tooltips=[('Is maximum','@isMax'),("Pad type","@typeF")]
p3 = figure(plot_width=700, plot_height=400, title="template")
widgets="query.name(), accordion.first(slider.P0(0,1,0.5,0,1),slider.commonF(0,15,5,0,5)),accordion.second(dropdown.MB(0,0.5,1)),accordion.second(checkbox.isMax(False)),slider.typeF(0,4,1,0)"

#xxx=bokehDraw(tree,"tailF>-1","tailF","slope","fraction",slider,p3,commonX=1,size=8,ncols=1,nEntries=100000000,tooltips=tooltips,variables=variables)
xxx=bokehDraw(tree,"tailF>=-1","tailF","slope","fraction",widgets,p3,commonX=1,size=8,ncols=1,nEntries=100000000,tooltips=tooltips)

# Convert selected variables from tree to panda and draw

In [ ]:
variables="pileUp:tailF:commonF:typeF:fraction:param.fElements[1]:norm:slope:isMax:MB:P0:P1:PA:isMaxTot"
df=treeToPanda(tree,variables,"1",100000000,0,".fElements");
#widgets="tab.first(slider.commonF(0,15,5,0,5)),tab.second(dropdown.MB(0,0.5,1)),accordion.first(slider.P0(0,1,0.5,0,1)),accordion.second(checkbox.isMax),slider.typeF(0,4,1,0)"
widgets="query.custom(), tab.sliders(slider.commonF(0,15,5,0,5),slider.fraction(0,100,4,0,100)),"
widgets+="tab.options(checkbox.P0(0),checkbox.P1(1),checkbox.isMax(True), checkbox.isMaxTot(false), checkbox.MB(), checkbox.pileUp()))"
bokeh=bokehDraw(df,"tailF>-1","tailF","slope","fraction",widgets,p3,commonX=1,size=6,ncols=1, tooltips=tooltips)

# Bokeh draw via figure array

In [ ]:
variables="pileUp:tailF:commonF:typeF:fraction:param.fElements[1]:norm:slope:isMax:MB:P0:P1:PA:isMaxTot"
df=treeToPanda(tree,variables,"1",100000000,0,".fElements");

tooltips=[('Is maximum','@isMax'),("Pad type","@typeF")]
p3 = figure(plot_width=700, plot_height=400, title="template")
widgets="query.custom(), accordion.first(slider.P0(0,1,0.5,0,1),slider.commonF(0,15,5,0,5)),accordion.second(dropdown.MB(0,0.5,1)),accordion.second(checkbox.isMax(False)),slider.typeF(0,4,1,0)"

figureArray= [
    [['tailF'], ['slope','norm-1'], {"color": "red", "size": 2, "colorZvar":"fraction"}],
    [['fraction'], ['slope','norm-1'], {"color": "red", "size": 2, "colorZvar":"tailF"}],
    ['table']
]
figureLayout: str = '((0,1),(2, x_visible=1),commonY=1, x_visible=1,y_visible=0,plot_height=250,plot_width=1000)'

In [ ]:
#bokeh=bokehDraw.fromArray(df,"tailF>-1",figureArray,widgets,layout=figureLayout, tooltips=tooltips)
fig=bokehDraw.fromArray(tree, "tailF>=-1", figureArray,widgets,tooltips=tooltips, layout=figureLayout,nEntries=100000000)